## <center> Hofstadter Butterfly </center> ##

We generate a fractal like-structure, called the Hofstadter Butterfly, which represents the energy
levels of  an electron travelling through a periodic lattice under the influence of a
magnetic field.

The mathematical model related to the  Hamiltonian of an electron in a two dimensional lattice,
subject to a perpendicular (uniform) magnetic field is the Almost Mathieu operator or Harper operator, which is 
a discrete one-dimensional Schrodinger operator that acts on the Hilbert space, $\ell^2(\mathbb{Z})$, of the infinite sequences. It is defined  by:

$$(H_{\Phi, K, \theta}u)_n=u_{n+1}+u_{n-1}+K\cos(n\Phi +\theta) u_n, \quad\Phi, K, \theta\in\mathbb{R}$$

When the magnetic flux  penetrating the lattice is a rational number, $\Phi=p/q$, with $p,q$  relatively prime integers,
 the spectrum of the above operator splits  into q subbands.  
 
 The representation of the operator spectrum as a
function of the rational flux exhibits a recursive structure with 
self-similarities. 

For a rational flux  $\Phi=2\pi p/q$, with $p, q$ relative prime integers, the potential $V_\theta(n)=K\cos(n\Phi+\theta)$
is periodic and the eigenvalue problem:

$$(H_{\Phi, K, \theta}u)_n=E u_n$$

reduces to a matrix eigenvalue problem associated to the following periodic Jacobi matrix, called Harper matrix:

$$
Ha(p, q, K, \theta)=\left(\begin{array}{ccccccc}K\cos(2\pi 0 p/q+\theta)&1 &0&\ldots&0& 0&1\\
                       1& K\cos(2\pi  p/q+\theta)&1&\ldots&0&0&0\\
                        \vdots&\vdots&\vdots&\ldots&\vdots&\vdots&\vdots\\
                         0&0&0&\ldots&1&K\cos(2\pi (q-2) p/q+\theta)&1\\
                         1&0&0&\ldots&0&1&K\cos(2\pi (q-1) p/q+\theta)\end{array}\right)$$
                         
  


The  scatter plot of all points of coordinates, $(p/q, E_n)$, with $p/q\in[0,1)$, $n=1,2, \ldots q$, and  $E_n$  running over the q eigenvalues of the Harper matrix   $Ha(p, q)$,
corresponding to $K=2$, $\theta=0$,  is called the Hofstadter butterfly, because it was generated first by Hofstadter in 1976.

For  any $q<qmax$ we should compute the eigenvalues of all matrices $Ha(p, q)$, with $p\in \{1, 2, \ldots q-1\}$, such that
$p, q$ are relatively prime numbers. But since $\cos$ is an odd $2\pi$-periodic function,  we have that   

$$\cos(2\pi n p/q)=\cos(2\pi n(q-p)/q),$$ and thus
 $$Ha(p, q)=Ha(q-p)$$.
 
 Hence  only the spectrum of the Harper matrices $Ha(p, q)$, with
$p\in\{1, 2, \ldots, q//2\}$, if $q$ is odd, respectively $p\in\{1, 2, \ldots, q/2-1\}$, if $q$ is even, will be calculated.

In [1]:
import platform
print(f'Python version: {platform.python_version()}')

Python version: 3.6.4


In [2]:
import plotly
plotly.__version__

'3.1.1'

In [1]:
import plotly.graph_objs as go
import numpy as np
from numpy import pi

In [2]:
def Gear(n): 
    ''' Generates the  Gear-type matrix, i.e. the periodic Jacobi matrix G=(0,..0;1,...1; +1)
    C.W. Gear, A simple set of test matrices for eigenvalue programs,
     Math. Comp. 23, 119-125.
     '''
    G=np.diag(np.ones(n - 1), -1) + np.diag(np.ones(n - 1), 1)
    
    G[0][n-1]=1
    G[n-1][0]=1
    return G


def eigs_Harper(p, q):
   
    d=[2*np.cos(2*np.pi*k*p/q) for k in range(q)] #define the diagonal of the Harper matrix   Ha(p,q)    
    Hd= np.diag(d)
    G = Gear(q)
    
    return list(np.linalg.eigvalsh(Hd+G))#eigenvalues of the Harper matrix 

def gcd(a, b): # Greatest Common Divisor
    if b == 0: return a
    return gcd(b, a % b)

In [3]:
def get_butterfly_points_even(qmax=101):# for  qmax=101 value we define 1036 irreducible fractions p/q, 
 
    phi=[]# the list of  of rational magnetic flux values, p/q
    E=[]# the list of energies
    text=[]# the list of hover strings

    for q in range(4, qmax, 2):    
        for p in range(1, q//2, 2):
            if gcd(p, q) == 1:
                phi.extend([p/q]*q+ [(q-p)/q]*q) #insert q copies of  p/q, respectively (q-p)/q, 
                                           #because the corresponding Harper matrix H(p,q), resp H(q-p, p), has q eigvals
                eigs_pq=eigs_Harper(p, q) 
                E.extend(eigs_pq*2)           
                p_text=[f"(p, q) = {(p,q)}"]*q+[f"(p,q) = {(q-p, q)}"]*q
                text.extend([f"{t}<br>E = {round(e, 3)}" for t, e in zip(p_text, eigs_pq*2)])
    return phi, E, text

In [4]:
def get_butterfly_points_odd(qmax=70):

    phi=[]
    E=[]
    text=[]

    for q in range(5, qmax, 1):    
        for p in range(1, q//2+1, 1):
            if gcd(p, q) == 1:
                phi.extend([p/q]*q+ [(q-p)/q]*q) 
                eigs_pq=eigs_Harper(p, q) 
                E.extend(eigs_pq*2)           
                p_text=[f"(p, q) = {(p,q)}"]*q+[f"(p,q) = {(q-p, q)}"]*q
                text.extend([f"{t}<br>E = {round(e, 3)}" for t, e in zip(p_text, eigs_pq*2)])
    return phi, E, text

In [5]:
def get_butterfly_trace(phi, E, text, color='blue', marker_size=1):
    return dict(type='scatter',
                x=phi,
                y=E,
                mode='markers',
                text=text,  
                marker=dict(color=color, size=marker_size),
                hoverinfo='text')

Generating the butterfly data,  with the function `get_butterfly_points_odd()` and then via `get_butterfly_points_even()`
we get two indistinguishable plots. 

The user can experiment plotting the Hofstadter butterfly associated to both data, succesively, or to their union. 

In [6]:
phi_e, E_e, text_e=get_butterfly_points_even(qmax=101)
#phi_o, E_o, text_o=get_butterfly_points_odd(qmax=70)

In [7]:
len(E_e)#points are plotted

69836

In [9]:
data=[get_butterfly_trace(phi_e, E_e, text_e)]

In [10]:
axis_style=dict(showline=True, 
                mirror=True, 
                zeroline=False, 
                showgrid=False, 
                ticklen=4)

In [11]:
layout=dict(title='Hofstadter butterfly',
            font=dict(family='Balto'),
            width=600, height=675,
            autosize=False,
            showlegend=False,
            xaxis=dict(axis_style, **dict( title='Phi (magnetic flux)', dtick=0.25)),
            yaxis=dict(axis_style, **dict( title='E (Energy)')),
            hovermode='closest')

In [12]:
fw=go.FigureWidget(data=data, layout=layout)

In [ ]:
fw # running this cell will insert the FigureWidget in the next one

To send the figure to Plotly cloud run the next cells:

In [13]:
import plotly.plotly as py

In [14]:
import warnings
warnings.filterwarnings("ignore")

In [18]:
py.sign_in('empet', 'api_key')
py.iplot(fw, filename='Hofstadter1')

The draw time for this plot will be slow for all clients.
